# Generate Gpt-2 Examples #

**prompt_text**:change the prompt to a text of your choice  
**response_length**: This is the maximum length of any response. It may choose to create shorter response itself. This cannot be longer than 1024  
**output_file**: This should be a .csv file. It can be downloaded an analyzed. If this is run multiple times, it will append all answers into the file.  
**num_of_responses**: This is usually a max of 4 for K80 GPU. It is possible that even at 4, the process may error. If this is the case rerun.  
  
**IN CASE OF ERROR**  
The process may error because the GPU gets filled up. Everytime you run this, make sure that the GPU starts with 0 memory used. The !Nvidia-smi command shows that.   
In the middle column it should say "0MB/11441MB"    
if it does not. The GPU needs to be cleared.  
Clear the GPU by clicking on the top menu "Kernel->Restart Kernel" Then rerun the notebook from the top cell.  

In [1]:

prompt_text="If God is defined as something that is all powerful and all knowing, a strong artificial intelligence might be an actual God. If this happens the implications for religion are"
#max reponse_length 1024
response_length=1000
#output_file will be created if it doesn't exist, otherwise answers will be appended
output_file="results.csv"
#max 4 (k80 gpu)
num_of_responses=4


In [28]:
#from ipyexperiments import *

In [29]:
!nvidia-smi


Tue Apr 14 13:07:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.64       Driver Version: 430.64       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    59W / 149W |  10078MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
all_seq=[]

In [3]:
#exp1=IPyExperimentsPytorch()

In [5]:
%%time
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

#device = xm.xla_device()
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = GPT2Tokenizer.from_pretrained('GPT2Model/GPT2Model/')
model = GPT2LMHeadModel.from_pretrained('GPT2Model/GPT2Model/')
model.to(device)
encoded_prompt=tokenizer.encode(prompt_text, add_special_tokens=True,return_tensors="pt")
encoded_prompt = encoded_prompt.to(device)

outputs = model.generate(encoded_prompt,response_length,temperature=.8,num_return_sequences=num_of_responses,repetition_penalty=85,do_sample=True,top_k=80,top_p=.85 )


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


KeyboardInterrupt: 

In [33]:
# Remove the batch dimension when returning multiple sequences
if len(outputs.shape) > 2:
    outputs.squeeze_()

In [34]:
generated_sequences=[]
total_sequence=""
for generated_sequence_idx, generated_sequence in enumerate(outputs):
        print("=== GENERATED SEQUENCE {} ===".format(generated_sequence_idx + 1))
        generated_sequence = generated_sequence.tolist()
        #print(generated_sequence)
        # Decode text
        text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
        #print(text)
        # Remove all text after the stop token
        stop_token='<|endoftext|>'
        text = text[: text.find(stop_token) if stop_token else None]

        # Add the prompt at the beginning of the sequence. Remove the excess text that was used for pre-processing
        total_sequence = (
            prompt_text + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]
        )

        generated_sequences.append(total_sequence)
        print(total_sequence)
        all_seq.append(total_sequence)

=== GENERATED SEQUENCE 1 ===
If God is defined as something that is all powerful and all knowing, a strong artificial intelligence might be an actual God. If this happens the implications for religion are severe: no one could ever know who or what was real before being given free will to choose it - hence we would have little need of religious institutions at which people believe they can receive some sort of knowledge about themselves through belief in their own "divine" souls (which most atheists accept). Instead there'd simply exist various human-invented religions based on nothing but shared experiences among humans; however shallow these beliefs may seem if you only listen within your personal bubble rather than taking them into account when talking with others outside yours! The idea here seems pretty straightforward though...


Of course many things don't fit neatly under our concept [of god]... It's hard enough trying make sense out loud without having someone else tell us exac

In [35]:
if len(all_seq)==len(set(all_seq)):
  print('no duplicates')
else:
  print('duplicates found')

no duplicates


In [36]:
import csv
import os
if os.path.exists(output_file):
    append_flag="a"
else: 
    append_flag="w"
with open (output_file, append_flag) as csvfile:
    writer=csv.writer(csvfile)
    for i in all_seq:
        writer.writerow([prompt_text, i])
    

In [37]:
print('run complete')

run complete
